In [1]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical   # For one hot encoding

2025-09-10 18:53:25.868287: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-10 18:53:25.868528: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-10 18:53:25.903161: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-10 18:53:26.724140: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# x_train[i] = the i-th image
# y_train[i] = the correct digit for that image

# x_train[0]

# y_train looks like [5, 0, 4, 1, 9, ...]
print(y_train.shape)
print(y_train)

(60000,)
[5 0 4 ... 5 6 8]


In [4]:
# Feature scaling

# 👉 The pixel values in MNIST images range from 0 to 255 (since each pixel is stored as an 8-bit integer).
# 0 = black
# 255 = white
# Values in between = shades of gray

x_train = x_train/255.0
x_test = x_test/255.0

In [5]:
# After normalizing this x_train
# x_train[0]

In [6]:
# One hot encoder to labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [7]:
# → 60,000 labels, but each label is now a vector of length 10.
# 5 → [0,0,0,0,0,1,0,0,0,0]
print(y_train.shape)
print(y_train)

(60000, 10)
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [8]:
model = Sequential([                           # Create a Sequential model (layers stacked one after another)
    Input(shape=(28, 28)),                     # Input layer: expects 28x28 grayscale images
    Flatten(),                                 # Flattens 28x28 into 784 values (1D vector)
    Dense(128, activation="relu"),             # Hidden layer: 128 neurons, ReLU adds non-linearity
    Dense(10, activation="softmax")            # Output layer: 10 neurons for digits 0–9, softmax gives probabilities
])



2025-09-10 18:53:27.758302: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
model.compile(
    optimizer="adam",                          # Optimizer: Adam adjusts weights during training efficiently
    loss="categorical_crossentropy",           # Loss function: compares predicted probabilities vs actual labels
    metrics=['accuracy']                       # Metric: track accuracy while training/testing
)

# This adjustment is done using an optimizer.
# Adam is one of the most popular optimizers.
# It automatically decides how big or small the weight updates should be for each neuron.

In [10]:
model.fit(x_train, y_train, epochs=5, batch_size=16)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9350 - loss: 0.2284
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9700 - loss: 0.0995
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9789 - loss: 0.0692
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9839 - loss: 0.0510
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9880 - loss: 0.0381


In [11]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"The test accuracy is {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9763 - loss: 0.0808   
The test accuracy is 0.9763000011444092


In [12]:
import cv2
import numpy as np

def preprocess_image(img_path):
    # Read as grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    # Invert if background is white
    if np.mean(img) > 127:
        img = cv2.bitwise_not(img)
    
    # Threshold the image (binary)
    _, img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    
    # Find contours to crop the digit
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(contours[0])
        img = img[y:y+h, x:x+w]
    
    # Resize while keeping aspect ratio
    h, w = img.shape
    if h > w:
        new_h = 20
        new_w = int(w * (20/h))
    else:
        new_w = 20
        new_h = int(h * (20/w))
    img = cv2.resize(img, (new_w, new_h))
    
    # Pad to 28x28
    pad_top = (28 - new_h) // 2
    pad_bottom = 28 - new_h - pad_top
    pad_left = (28 - new_w) // 2
    pad_right = 28 - new_w - pad_left
    img = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right,
                             cv2.BORDER_CONSTANT, value=0)
    
    # Normalize
    img = img / 255.0
    img = np.expand_dims(img, axis=0)   # shape (1, 28, 28)
    
    return img

# Use it
img = preprocess_image("6_image.png")
prediction = model.predict(img)
predicted_class = np.argmax(prediction)
print("Predicted digit:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted digit: 6


In [13]:
# Correct only for MNIST-like images with proper padding like the MNIST dataset.
import cv2
import numpy as np

img_path = "2_image.png"  # your file

# Read as grayscale
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Invert colors: black digit → white digit
img = cv2.bitwise_not(img)

# Resize to 28x28 (like MNIST)
img = cv2.resize(img, (28, 28))

# Normalize
img = img / 255.0

# Reshape for model input
img = np.expand_dims(img, axis=0)  # shape (1, 28, 28)

# Predict
prediction = model.predict(img)
predicted_class = np.argmax(prediction)

print("Predicted digit:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted digit: 2
